# Transcript Analysis

In [ ]:
import json
import pandas as pd
import numpy as np
from nltk.tokenize import word_tokenize, wordpunct_tokenize
from nltk.corpus import stopwords 
import nltk 
from gensim.models import KeyedVectors
import gensim.downloader as api

In [ ]:
f = open("Data/2005_codes.json",encoding='utf-8')
json2005=json.load(f)

In [ ]:
mt = pd.read_csv("medicaltranscriptions.csv")



```
# This is formatted as code
```


>## Build Model




In [ ]:
dic = open('PubMed-and-PMC-w2v.bin', 'rb')
from gensim.models import KeyedVectors
model = KeyedVectors.load_word2vec_format(dic, binary=True)

In [ ]:
key_vec=np.empty((0, 200), float)

for i in range(len(mt["keywords"])):
    word2vec_ls=np.empty((0, 200), float)
    
    if type(mt["keywords"][i]) == str:
        mt_token_pos=nltk.pos_tag(nltk.word_tokenize(mt["keywords"][i]))
    else:
        mt_token_pos=nltk.pos_tag(nltk.word_tokenize(mt["transcription"][i]))
    
    for word, pos in mt_token_pos:
        if (pos == 'NN' or pos == 'NNP' or pos == 'NNS' or pos == 'NNPS'):
            try:
                word2vec_ls=np.vstack([word2vec_ls, model.word_vec(word)])
            except:
                pass
    if len(word2vec_ls)==0:
        k_vec=np.zeros((1, 200))
    else:
        k_vec = np.mean(word2vec_ls, axis=0).reshape(1, 200)
    key_vec=np.vstack([key_vec, k_vec])

In [ ]:
key_rep=pd.DataFrame(key_vec)

In [ ]:
ICD = pd.read_excel("allvalid2011 (detailed titles headings).xls")
ICD = ICD.drop([0,1,2,3,4,5],axis = 0).reset_index()
ICD = ICD.drop(["index","Unnamed: 0"],axis = 1)
ICD.columns = ["code","title"]

In [ ]:
droplist = []
for i in range(ICD.shape[0]): 
    if "-" in ICD.loc[i,"code"] or "." in ICD.loc[i,"code"]:
        droplist.append(i) 

In [ ]:
ICD = ICD.drop(droplist,axis = 0).reset_index(drop = True)

In [ ]:
ICD_vec=np.empty((0, 200), float)

for i in range(len(ICD["title"])):
    word2vec_ls=np.empty((0, 200), float)
    mt_token_pos=nltk.pos_tag(nltk.word_tokenize(ICD["title"][i]))
    for word, pos in mt_token_pos:
        if (pos == 'NN' or pos == 'NNP' or pos == 'NNS' or pos == 'NNPS'):
            try:
                word2vec_ls=np.vstack([word2vec_ls, model.word_vec(word)])
            except:
                pass
    if len(word2vec_ls)==0:
        code_vec=np.zeros((1, 200))
    else:
        code_vec = np.mean(word2vec_ls, axis=0).reshape(1, 200)
    ICD_vec=np.vstack([ICD_vec, code_vec])

In [ ]:
ICD_rep=pd.DataFrame(ICD_vec)

In [ ]:
key_rep.shape

(4999, 200)

In [ ]:
ICD_rep.shape

(1862, 200)

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
ICD_assign = None
for i in range(key_rep.shape[0]):
    max_sim = 0
    ICD_index = 0
    for j in range(ICD_rep.shape[0]):
        sim = cosine_similarity([key_rep.loc[i,:], ICD_rep.loc[j,:]])[0,1]
        if sim >= max_sim:
            max_sim = sim
            ICD_index = j
    ICD_assign = pd.concat([ICD_assign,pd.DataFrame([i,max_sim,ICD_index]).T], axis =0)

In [ ]:
ICD_assign_1 = ICD_assign.reset_index(drop = True)
ICD_assign_1.columns = ["transcript","similarity","ICD"]

In [ ]:
def code(i):
    return ICD.loc[i,"code"]
ICD_assign_1["ICD"] = ICD_assign_1["ICD"].apply(code)

In [ ]:
ICD_assign_2 = ICD_assign_1[ICD_assign_1["similarity"]>=0.7]

In [ ]:
ICD_assign_2.shape

(1840, 3)

In [ ]:
NounsTable=pd.DataFrame({"Key":list(ICD_assign_2["ICD"]), "Occurence": np.ones(len(list(ICD_assign_2["ICD"])))})
Nouncount=NounsTable.groupby(['Key'], sort=True).Occurence.sum().reset_index()
Nouncount = Nouncount.sort_values(by = 'Occurence', ascending=[False])

In [ ]:
Nouncount.head()

,Key,Occurence
195,S32,106.0
106,K56,56.0
241,Y60,50.0
104,K46,47.0
214,S73,44.0


In [ ]:
ICD_assign_2[ICD_assign_2["ICD"] == "Y60"].sort_index(axis = 0, ascending = False, by = ["similarity"]).head()

D:\Anaconda\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: by argument to sort_index is deprecated, please use .sort_values(by=...)
  """Entry point for launching an IPython kernel.


,transcript,similarity,ICD
1183,1183.0,0.788644,Y60
3675,3675.0,0.782450,Y60
3745,3745.0,0.763244,Y60
716,716.0,0.760833,Y60
2949,2949.0,0.756799,Y60


In [ ]:
# Using certain disease codes
ICD_1 = ICD[((ICD["code"] >= "I00") & (ICD["code"] <= "I99")) | ((ICD["code"] >= "C00") & (ICD["code"] <= "C97")) | ((ICD["code"] >= "J00") & (ICD["code"] <= "J98")) | ((ICD["code"] >= "E00") & (ICD["code"] <= "E90")) | ((ICD["code"] >= "G00") & (ICD["code"] <= "G98"))].reset_index()

In [ ]:
ICD_1.shape

(340, 3)

In [ ]:
ICD_1.head()

,index,code,title
0,168,C00,Malignant neoplasm of lip
1,169,C01,Malignant neoplasm of base of tongue
2,170,C02,Malignant neoplasm of other and unspecified pa...
3,171,C03,Malignant neoplasm of gum
4,172,C04,Malignant neoplasm of floor of mouth


In [ ]:
ICD_vec_1=np.empty((0, 200), float)

for i in range(len(ICD_1["title"])):
    word2vec_ls=np.empty((0, 200), float)
    mt_token_pos=nltk.pos_tag(nltk.word_tokenize(ICD_1["title"][i]))
    for word, pos in mt_token_pos:
        if (pos == 'NN' or pos == 'NNP' or pos == 'NNS' or pos == 'NNPS'):
            try:
                word2vec_ls=np.vstack([word2vec_ls, model.word_vec(word)])
            except:
                pass
    if len(word2vec_ls)==0:
        code_vec=np.zeros((1, 200))
    else:
        code_vec = np.mean(word2vec_ls, axis=0).reshape(1, 200)
    ICD_vec_1=np.vstack([ICD_vec_1, code_vec])

ICD_rep_1=pd.DataFrame(ICD_vec_1)

In [ ]:
ICD_assign_main = None
for i in range(key_rep.shape[0]):
    max_sim = 0
    ICD_index = 0
    for j in range(ICD_rep_1.shape[0]):
        sim = cosine_similarity([key_rep.loc[i,:], ICD_rep_1.loc[j,:]])[0,1]
        if sim >= max_sim:
            max_sim = sim
            ICD_index = j
    ICD_assign_main = pd.concat([ICD_assign_main,pd.DataFrame([i,max_sim,ICD_index]).T], axis =0)

In [ ]:
ICD_assign_main_1 = ICD_assign_main.reset_index(drop = True)
ICD_assign_main_1.columns = ["transcript","similarity","ICD"]

In [ ]:
def code(i):
    return ICD_1.loc[i,"code"]
ICD_assign_main_1["ICD"] = ICD_assign_main_1["ICD"].apply(code)

In [ ]:
ICD_assign_main_1.shape

(4999, 3)

In [ ]:
ICD_assign_main_2 = ICD_assign_main_1[ICD_assign_main_1["similarity"]>=0.7]
ICD_assign_main_2.shape

(533, 3)

In [ ]:
NounsTable_main=pd.DataFrame({"Key":list(ICD_assign_main_2["ICD"]), "Occurence": np.ones(len(list(ICD_assign_main_2["ICD"])))})
Nouncount_main=NounsTable_main.groupby(['Key'], sort=True).Occurence.sum().reset_index()
Nouncount_main = Nouncount_main.sort_values(by = 'Occurence', ascending=[False])

In [ ]:
Nouncount_main.head()

,Key,Occurence
79,I81,74.0
76,I71,53.0
81,I85,52.0
77,I72,39.0
50,G57,38.0




## Check certain examples



In [ ]:
# Check certain examples
ICD_assign_main_2[ICD_assign_main_2["ICD"] == "I81"].sort_index(axis = 0, ascending = False, by = ["similarity"]).head()

D:\Anaconda\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: by argument to sort_index is deprecated, please use .sort_values(by=...)
  """Entry point for launching an IPython kernel.


,transcript,similarity,ICD
4673,4673.0,0.899610,I81
362,362.0,0.882048,I81
3051,3051.0,0.863330,I81
1190,1190.0,0.854879,I81
3063,3063.0,0.817132,I81


In [ ]:
mt_word = []
stop_words = set(stopwords.words('english')) 
for i in range(len(mt["keywords"])):
    
    if type(mt["keywords"][i]) == str:
        word_tokens = wordpunct_tokenize(mt["keywords"][i])
    else:
        word_tokens = wordpunct_tokenize(mt["transcription"][i])
        
    word_tokens = [w.lower() for w in word_tokens] 
    word_tokens = [w.lower() for w in word_tokens if not w in stop_words] 
    word_tokens = [w.lower() for w in word_tokens if w.isalpha()] 
    mt_token_pos=nltk.pos_tag(word_tokens)
    
    wordlist = []
    for word, pos in mt_token_pos:
        if (pos == 'NN' or pos == 'NNP' or pos == 'NNS' or pos == 'NNPS'):
            wordlist.append(word)
    mt_word.append(wordlist)

In [ ]:
ICD_word = []
for i in range(len(ICD_1["title"])):
    word_tokens = wordpunct_tokenize(ICD_1["title"][i])
    word_tokens = [w.lower() for w in word_tokens] 
    word_tokens = [w.lower() for w in word_tokens if not w in stop_words] 
    word_tokens = [w.lower() for w in word_tokens if w.isalpha()] 
    ICD_token_pos=nltk.pos_tag(word_tokens)
    
    wordlist = []
    for word, pos in ICD_token_pos:
        if (pos == 'NN' or pos == 'NNP' or pos == 'NNS' or pos == 'NNPS'):
            wordlist.append(word)
    ICD_word.append(wordlist)



## Try Jaccard Similarity



In [ ]:
# try jaccard similarity
def jaccard(text1, text2):
    wordset1 = set(text1)
    wordset2 = set(text2)
    inter = wordset1 & wordset2
    union = wordset1 | wordset2
    count = len(inter)
    jaccard = count / (len(union) + 1)
    return jaccard

In [ ]:
ICD_assign_j = None
for i in range(len(mt_word)):
    max_sim = 0
    ICD_index = 0
    for j in range(len(ICD_word)):
        sim = jaccard(mt_word[i], ICD_word[j])
        if sim >= max_sim:
            max_sim = sim
            ICD_index = j
    ICD_assign_j = pd.concat([ICD_assign_j,pd.DataFrame([i,max_sim,ICD_index]).T], axis =0)

In [ ]:
ICD_assign_j = ICD_assign_j.reset_index(drop = True)
ICD_assign_j.columns = ["transcript","similarity","ICD"]
def code(i):
    return ICD_1.loc[i,"code"]
ICD_assign_j["ICD"] = ICD_assign_j["ICD"].apply(code)

In [ ]:
ICD_assign_j_1 = ICD_assign_j[ICD_assign_j["similarity"]>=0.1]
ICD_assign_j_1.shape

(931, 3)

In [ ]:
NounsTable_j=pd.DataFrame({"Key":list(ICD_assign_j_1["ICD"]), "Occurence": np.ones(len(list(ICD_assign_j_1["ICD"])))})
Nouncount_j=NounsTable_j.groupby(['Key'], sort=True).Occurence.sum().reset_index()
Nouncount_j = Nouncount_j.sort_values(by = 'Occurence', ascending=[False])

In [ ]:
Nouncount_j.head()

,Key,Occurence
118,J44,59.0
88,I50,43.0
45,E24,33.0
68,G54,31.0
83,I44,30.0


In [ ]:
ICD_assign_j_1[ICD_assign_j_1["ICD"] == "J44"].sort_index(axis = 0, ascending = False, by = ["similarity"]).head()

D:\Anaconda\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: by argument to sort_index is deprecated, please use .sort_values(by=...)
  """Entry point for launching an IPython kernel.


,transcript,similarity,ICD
3060,3060.0,0.200000,J44
3976,3976.0,0.166667,J44
4022,4022.0,0.166667,J44
3309,3309.0,0.166667,J44
2756,2756.0,0.166667,J44




## Try Use Transcription Instead of Keywords



In [ ]:
# try use transcription instead of keywords
key_vec_1=np.empty((0, 200), float)

for i in range(len(mt["keywords"])):
    word2vec_ls=np.empty((0, 200), float)
    if type(mt["transcription"][i]) == str:
        mt_token_pos=nltk.pos_tag(nltk.word_tokenize(mt["transcription"][i]))
    else:
        mt_token_pos=nltk.pos_tag(nltk.word_tokenize(mt["keywords"][i]))
    
    for word, pos in mt_token_pos:
        if (pos == 'NN' or pos == 'NNP' or pos == 'NNS' or pos == 'NNPS'):
            try:
                word2vec_ls=np.vstack([word2vec_ls, model.word_vec(word)])
            except:
                pass
    if len(word2vec_ls)==0:
        k_vec_1=np.zeros((1, 200))
    else:
        k_vec_1 = np.mean(word2vec_ls, axis=0).reshape(1, 200)
    key_vec_1=np.vstack([key_vec_1, k_vec_1])

In [ ]:
key_rep_1=pd.DataFrame(key_vec_1)

In [ ]:
ICD_assign_main_t = None
for i in range(key_rep.shape[0]):
    max_sim = 0
    ICD_index = 0
    for j in range(ICD_rep_1.shape[0]):
        sim = cosine_similarity([key_rep_1.loc[i,:], ICD_rep_1.loc[j,:]])[0,1]
        if sim >= max_sim:
            max_sim = sim
            ICD_index = j
    ICD_assign_main_t = pd.concat([ICD_assign_main_t,pd.DataFrame([i,max_sim,ICD_index]).T], axis =0)

In [ ]:
ICD_assign_main_t_1 = ICD_assign_main_t.reset_index(drop = True)
ICD_assign_main_t_1.columns = ["transcript","similarity","ICD"]

In [ ]:
def code(i):
    return ICD_1.loc[i,"code"]
ICD_assign_main_t_1["ICD"] = ICD_assign_main_t_1["ICD"].apply(code)

In [ ]:
ICD_assign_main_t_1.shape

(4999, 3)

In [ ]:
ICD_assign_main_t_2 = ICD_assign_main_t_1[ICD_assign_main_t_1["similarity"]>=0.7]
ICD_assign_main_t_2.shape

(141, 3)

In [ ]:
NounsTable_main_t=pd.DataFrame({"Key":list(ICD_assign_main_t_2["ICD"]), "Occurence": np.ones(len(list(ICD_assign_main_t_2["ICD"])))})
Nouncount_main_t=NounsTable_main_t.groupby(['Key'], sort=True).Occurence.sum().reset_index()
Nouncount_main_t = Nouncount_main_t.sort_values(by = 'Occurence', ascending=[False])

In [ ]:
Nouncount_main_t.head()

,Key,Occurence
15,I71,27.0
12,G57,15.0
18,J85,13.0
17,I81,10.0
16,I72,10.0


In [ ]:
ICD = pd.read_excel("allvalid2011 (detailed titles headings).xls")
ICD = ICD.drop([0,1,2,3,4,5],axis = 0).reset_index()
ICD = ICD.drop(["index","Unnamed: 0"],axis = 1)
ICD.columns = ["code","title"]

In [ ]:
droplist = []
for i in range(ICD.shape[0]): 
    if "-" in ICD.loc[i,"code"]:
        droplist.append(i) 
ICD = ICD.drop(droplist,axis = 0).reset_index(drop = True)



## Try Converting Codes With More Detialed Forms Into Higher Level codes



In [ ]:
# try converting codes with more detialed forms (.xx) into higher level codes
j = 0
for i in range(ICD.shape[0]):
    if "." not in ICD.loc[i,"code"]:
        j = i
    else:
        ICD.loc[j,"title"] = ICD.loc[j,"title"] + " " + ICD.loc[i,"title"]

In [ ]:
droplist_1 = []
for i in range(ICD.shape[0]): 
    if "." in ICD.loc[i,"code"]:
        droplist_1.append(i) 
ICD = ICD.drop(droplist_1,axis = 0).reset_index(drop = True)

In [ ]:
ICD_1 = ICD[((ICD["code"] >= "I00") & (ICD["code"] <= "I99")) | ((ICD["code"] >= "C00") & (ICD["code"] <= "C97")) | ((ICD["code"] >= "J00") & (ICD["code"] <= "J98")) | ((ICD["code"] >= "E00") & (ICD["code"] <= "E90")) | ((ICD["code"] >= "G00") & (ICD["code"] <= "G98"))].reset_index()

In [ ]:
ICD_1.head()

,index,code,title
0,168,C00,Malignant neoplasm of lip Malignant neoplasm o...
1,169,C01,Malignant neoplasm of base of tongue
2,170,C02,Malignant neoplasm of other and unspecified pa...
3,171,C03,Malignant neoplasm of gum Malignant neoplasm o...
4,172,C04,Malignant neoplasm of floor of mouth Malignant...


In [ ]:
ICD_vec_1=np.empty((0, 200), float)

for i in range(len(ICD_1["title"])):
    word2vec_ls=np.empty((0, 200), float)
    mt_token_pos=nltk.pos_tag(nltk.word_tokenize(ICD_1["title"][i]))
    for word, pos in mt_token_pos:
        if (pos == 'NN' or pos == 'NNP' or pos == 'NNS' or pos == 'NNPS'):
            try:
                word2vec_ls=np.vstack([word2vec_ls, model.word_vec(word)])
            except:
                pass
    if len(word2vec_ls)==0:
        code_vec=np.zeros((1, 200))
    else:
        code_vec = np.mean(word2vec_ls, axis=0).reshape(1, 200)
    ICD_vec_1=np.vstack([ICD_vec_1, code_vec])

ICD_rep_1=pd.DataFrame(ICD_vec_1)

In [ ]:
ICD_assign_main_m = None
for i in range(key_rep.shape[0]):
    max_sim = 0
    ICD_index = 0
    for j in range(ICD_rep_1.shape[0]):
        sim = cosine_similarity([key_rep.loc[i,:], ICD_rep_1.loc[j,:]])[0,1]
        if sim >= max_sim:
            max_sim = sim
            ICD_index = j
    ICD_assign_main_m = pd.concat([ICD_assign_main_m,pd.DataFrame([i,max_sim,ICD_index]).T], axis =0)

In [ ]:
ICD_assign_main_m_1 = ICD_assign_main_m.reset_index(drop = True)
ICD_assign_main_m_1.columns = ["transcript","similarity","ICD"]

In [ ]:
def code(i):
    return ICD_1.loc[i,"code"]
ICD_assign_main_m_1["ICD"] = ICD_assign_main_m_1["ICD"].apply(code)

In [ ]:
ICD_assign_main_m_2 = ICD_assign_main_m_1[ICD_assign_main_m_1["similarity"]>=0.7]
ICD_assign_main_m_2.shape

(714, 3)

In [ ]:
NounsTable_main_m=pd.DataFrame({"Key":list(ICD_assign_main_m_2["ICD"]), "Occurence": np.ones(len(list(ICD_assign_main_m_2["ICD"])))})
Nouncount_main_m=NounsTable_main_m.groupby(['Key'], sort=True).Occurence.sum().reset_index()
Nouncount_main_m = Nouncount_main_m.sort_values(by = 'Occurence', ascending=[False])

In [ ]:
Nouncount_main_m.head(10)

,Key,Occurence
57,I72,110.0
30,G56,47.0
45,I36,31.0
31,G57,30.0
2,C20,26.0
62,I85,26.0
59,I81,23.0
27,G44,21.0
12,C64,20.0
63,I89,17.0


In [ ]:
ICD_assign_main_m_2[ICD_assign_main_m_2["ICD"] == "I72"].sort_index(axis = 0, ascending = False, by = ["similarity"]).head()

D:\Anaconda\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: by argument to sort_index is deprecated, please use .sort_values(by=...)
  """Entry point for launching an IPython kernel.


,transcript,similarity,ICD
3569,3569.0,0.898108,I72
852,852.0,0.897468,I72
4970,4970.0,0.849942,I72
1268,1268.0,0.849942,I72
4882,4882.0,0.849262,I72


In [ ]:
ICD = pd.read_excel("allvalid2011 (detailed titles headings).xls")
ICD = ICD.drop([0,1,2,3,4,5],axis = 0).reset_index()
ICD = ICD.drop(["index","Unnamed: 0"],axis = 1)
ICD.columns = ["code","title"]



## Conclude detailed ICD forms



In [ ]:
# Conclude detailed ICD forms
droplist = []
for i in range(ICD.shape[0]): 
    if "-" in ICD.loc[i,"code"]:
        droplist.append(i) 
ICD = ICD.drop(droplist,axis = 0).reset_index(drop = True)

In [ ]:
mergelist = []
for i in range(ICD.shape[0]): 
    if "I" in ICD.loc[i,"code"] or "C" in ICD.loc[i,"code"] or "J" in ICD.loc[i,"code"] or "E" in ICD.loc[i,"code"] or "G" in ICD.loc[i,"code"]:
        mergelist.append(i) 
ICD_2 = ICD.loc[mergelist,:].reset_index(drop = True)

In [ ]:
ICD_2.shape

(1900, 2)

In [ ]:
ICD_vec_2=np.empty((0, 200), float)

for i in range(len(ICD_2["title"])):
    word2vec_ls=np.empty((0, 200), float)
    mt_token_pos=nltk.pos_tag(nltk.word_tokenize(ICD_2["title"][i]))
    for word, pos in mt_token_pos:
        if (pos == 'NN' or pos == 'NNP' or pos == 'NNS' or pos == 'NNPS'):
            try:
                word2vec_ls=np.vstack([word2vec_ls, model.word_vec(word)])
            except:
                pass
    if len(word2vec_ls)==0:
        code_vec=np.zeros((1, 200))
    else:
        code_vec = np.mean(word2vec_ls, axis=0).reshape(1, 200)
    ICD_vec_2=np.vstack([ICD_vec_2, code_vec])

ICD_rep_2=pd.DataFrame(ICD_vec_2)

In [ ]:
ICD_assign_main_d = None
for i in range(key_rep.shape[0]):
    max_sim = 0
    ICD_index = 0
    for j in range(ICD_rep_2.shape[0]):
        sim = cosine_similarity([key_rep.loc[i,:], ICD_rep_2.loc[j,:]])[0,1]
        if sim >= max_sim:
            max_sim = sim
            ICD_index = j
    ICD_assign_main_d = pd.concat([ICD_assign_main_d,pd.DataFrame([i,max_sim,ICD_index]).T], axis =0)

In [ ]:
ICD_assign_main_d_1 = ICD_assign_main_d.reset_index(drop = True)
ICD_assign_main_d_1.columns = ["transcript","similarity","ICD"]

In [ ]:
def code(i):
    return ICD_2.loc[i,"code"]
ICD_assign_main_d_1["ICD"] = ICD_assign_main_d_1["ICD"].apply(code)

In [ ]:
ICD_assign_main_d_2 = ICD_assign_main_d_1[ICD_assign_main_d_1["similarity"]>=0.7]
ICD_assign_main_d_2.shape

(1291, 3)

In [ ]:
NounsTable_main_d=pd.DataFrame({"Key":list(ICD_assign_main_d_2["ICD"]), "Occurence": np.ones(len(list(ICD_assign_main_d_2["ICD"])))})
Nouncount_main_d=NounsTable_main_d.groupby(['Key'], sort=True).Occurence.sum().reset_index()
Nouncount_main_d = Nouncount_main_d.sort_values(by = 'Occurence', ascending=[False])

In [ ]:
Nouncount_main_d.head(10)

,Key,Occurence
123,I25.4,83.0
29,C41.3,53.0
150,I63.5,47.0
41,C47.1,45.0
32,C43.7,34.0
101,G44.2,30.0
14,C18.7,26.0
26,C40.0,25.0
104,G50.1,23.0
70,C76.2,22.0


In [ ]:
ICD_assign_main_d_2[ICD_assign_main_d_2["ICD"] == "G44.2"].sort_index(axis = 0, ascending = False, by = ["similarity"]).tail(10)

D:\Anaconda\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: by argument to sort_index is deprecated, please use .sort_values(by=...)
  """Entry point for launching an IPython kernel.


,transcript,similarity,ICD
1376,1376.0,0.722336,G44.2
4495,4495.0,0.720718,G44.2
2460,2460.0,0.720414,G44.2
1378,1378.0,0.719007,G44.2
3526,3526.0,0.717281,G44.2
4360,4360.0,0.717281,G44.2
3234,3234.0,0.712694,G44.2
1751,1751.0,0.712651,G44.2
2952,2952.0,0.712233,G44.2
4213,4213.0,0.707393,G44.2
